<a href="https://colab.research.google.com/github/olafthiele/oop22/blob/main/OOP_22_1_H%C3%B6ren_Speech_To_Text_wav2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchaudio ipywebrtc
!sudo apt update
!sudo apt install ffmpeg
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 260 kB 5.5 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-securit

In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio
from google.colab import output
output.enable_custom_widget_manager()

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [ ]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav file.wav -y -hide_banner -loglevel panic
sig, sr = torchaudio.load("file.wav")
print(sig.shape)
Audio(data=sig, rate=sr)



torch.Size([1, 172800])


In [ ]:
!wget -O - http://ainblick.de/test/wavs.tar.gz | tar xvfz -
!cp german.wav speech.wav

sig, sr = torchaudio.load("speech.wav")
Audio(data=sig, rate=sr)

--2022-01-28 20:19:22--  http://ainblick.de/test/wavs.tar.gz
Resolving ainblick.de (ainblick.de)... 81.169.222.220
Connecting to ainblick.de (ainblick.de)|81.169.222.220|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ainblick.de/test/wavs.tar.gz [following]
--2022-01-28 20:19:23--  https://ainblick.de/test/wavs.tar.gz
Connecting to ainblick.de (ainblick.de)|81.169.222.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 265514 (259K) [application/x-gzip]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               german.wav
-                    15%[==>                 ]  40.00K   177KB/s               english.wav
-                   100%[===================>] 259.29K   574KB/s    in 0.5s    

2022-01-28 20:19:24 (574 KB/s) - written to stdout [265514/265514]



In [ ]:
!pip install asrecognition

In [ ]:
from asrecognition import ASREngine

asr = ASREngine("de", model_path="jonatasgrosman/wav2vec2-large-xlsr-53-german")

audio_paths = ["file.wav", "speech.wav"]
transcriptions = asr.transcribe(audio_paths)

transcriptions

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:354: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


[{'path': 'file.wav', 'transcription': 'ICH BIN HUNGRIG'},
 {'path': 'speech.wav', 'transcription': 'DIE AUSSICHTEN SONNIG'}]

In [ ]:
!pip install transformers
!pip install pyctcdecode

In [ ]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM
import warnings
warnings.filterwarnings('ignore')

model_id = "jonatasgrosman/wav2vec2-large-xlsr-53-german"
processor = Wav2Vec2Processor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)# .to("cuda")
audio, rate = torchaudio.load("speech.wav")
input_values = processor(audio, return_tensors = "pt", sampling_rate=rate).input_values
logits = None
with torch.no_grad():
    # logits = model(input_values.to("cuda")).logits
    logits = model(input_values[0]).logits
predicted_ids = torch.argmax(logits, dim=-1)
print(predicted_ids)
transcription = processor.batch_decode(predicted_ids)

transcription

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0, 15,  0, 11, 11, 11,
          0,  0,  0,  0,  0,  4,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  7,  0,  0, 27,  0,  0,  0,  0,  0,  0,
         25,  0,  0,  0,  0,  0,  0,  0, 25,  0,  0, 15,  0,  0,  0,  9,  9, 14,
         14,  0,  0,  0,  0, 26,  0,  0,  0,  0, 11,  0,  0,  0,  0,  0, 20,  0,
          0,  0,  0,  4,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 25,  0,  0,  0, 21,  0,  0,
         20,  0,  0,  0, 20,  0,  0,  0, 15,  0,  0,  0,  0, 13,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4]])


['DIE AUSSICHTEN SONNIG']

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM

model_id = "flozi00/wav2vec2-large-xlsr-53-german-with-lm"
processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)
transcription = processor.batch_decode(logits.detach().cpu().numpy()).text

ImportError: ignored